# Deep Learning

## Assignment 3

Previously in 2_fullyconnected.ipynb, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.


In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in 1_notmnist.ipynb.


In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:

   data as a flat matrix,
   
   labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

## Problem 1

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor t using nn.l2_loss(t). The right amount of regularization should improve your validation / test accuracy.

-- Adding regularization to logistic regression model:

In [5]:
batch_size = 128
beta = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases

  ''' 
  Here is the difference with the code in Assignment#2. 
  pay attention that the reggularization should be applied on weights only.

  '''
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)+
    beta * tf.nn.l2_loss(weights))


  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 45.889393
Minibatch accuracy: 10.9%
Validation accuracy: 14.8%
Minibatch loss at step 500: 0.739927
Minibatch accuracy: 85.9%
Validation accuracy: 81.6%
Minibatch loss at step 1000: 0.800279
Minibatch accuracy: 80.5%
Validation accuracy: 81.3%
Minibatch loss at step 1500: 0.565932
Minibatch accuracy: 83.6%
Validation accuracy: 81.1%
Minibatch loss at step 2000: 0.648359
Minibatch accuracy: 88.3%
Validation accuracy: 81.2%
Minibatch loss at step 2500: 0.782483
Minibatch accuracy: 78.9%
Validation accuracy: 81.1%
Minibatch loss at step 3000: 0.784392
Minibatch accuracy: 80.5%
Validation accuracy: 81.4%
Test accuracy: 88.6%


-- Adding regularization to neural network model:

In [7]:
batch_size = 128
num_nodes = 1024 #number of nodes in the hidden layer

graph = tf.Graph()
with graph.as_default():

# Input data. For the training data, we use a placeholder that will be fed
# at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  def weight_variable(shape):
    initial = tf.truncated_normal(shape)
    return tf.Variable(initial)

  def bias_variable(shape):
    initial = tf.zeros(shape)
    return tf.Variable(initial) 

  def nn_layer(input_tensor, input_dim, output_dim, act=tf.nn.relu):
    """  creates a simple neural net layer."""
    weights = weight_variable([input_dim, output_dim])
    biases = bias_variable([output_dim])
    preactivate = tf.matmul(input_tensor, weights) + biases
    activations = act(preactivate)
    return weights, biases, activations

  weights1, biases1, hidden1 = nn_layer(tf_train_dataset, image_size * image_size, num_nodes)
  weights2, biases2, output = nn_layer(hidden1, num_nodes, num_labels, act=tf.identity)

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=output)+
    beta * tf.nn.l2_loss(weights1) + beta * tf.nn.l2_loss(weights2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(output)

  valid_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_hidden, weights2) + biases2)

  test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  test_prediction = tf.nn.softmax(tf.matmul(test_hidden, weights2) + biases2)

In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3500.500977
Minibatch accuracy: 7.0%
Validation accuracy: 26.2%
Minibatch loss at step 500: 21.226160
Minibatch accuracy: 88.3%
Validation accuracy: 84.0%
Minibatch loss at step 1000: 0.949400
Minibatch accuracy: 82.0%
Validation accuracy: 83.2%
Minibatch loss at step 1500: 0.584956
Minibatch accuracy: 88.3%
Validation accuracy: 83.6%
Minibatch loss at step 2000: 0.610904
Minibatch accuracy: 89.8%
Validation accuracy: 83.4%
Minibatch loss at step 2500: 0.715976
Minibatch accuracy: 83.6%
Validation accuracy: 83.2%
Minibatch loss at step 3000: 0.764885
Minibatch accuracy: 84.4%
Validation accuracy: 83.6%
Test accuracy: 90.3%


## Problem 2

Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

In [9]:
num_steps = 3001

''' we select a small subset from training dataset'''
train_dataset_2 = train_dataset[:500, :]
train_labels_2 = train_labels[:500]

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels_2.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset_2[offset:(offset + batch_size), :]
        batch_labels = train_labels_2[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step {}: {}".format(step, l))
            print("Minibatch accuracy: {:.1f}".format(accuracy(predictions, batch_labels)))
            print("Validation accuracy: {:.1f}".format(accuracy(valid_prediction.eval(), valid_labels)))
    print("Test accuracy: {:.1f}".format(accuracy(test_prediction.eval(), test_labels)))

Initialized
Minibatch loss at step 0: 3554.41137695
Minibatch accuracy: 7.8
Validation accuracy: 25.5
Minibatch loss at step 500: 21.0351867676
Minibatch accuracy: 100.0
Validation accuracy: 77.7
Minibatch loss at step 1000: 0.491449594498
Minibatch accuracy: 99.2
Validation accuracy: 79.3
Minibatch loss at step 1500: 0.300125807524
Minibatch accuracy: 100.0
Validation accuracy: 79.3
Minibatch loss at step 2000: 0.281628400087
Minibatch accuracy: 100.0
Validation accuracy: 79.4
Minibatch loss at step 2500: 0.280457288027
Minibatch accuracy: 100.0
Validation accuracy: 79.4
Minibatch loss at step 3000: 0.276854097843
Minibatch accuracy: 100.0
Validation accuracy: 79.4
Test accuracy: 86.6


It can be seen that we got 100% aauracy with Minibatch accuracy but our test accuracy dropped from 90.4% to 86.5%.


## Problem 3

Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides nn.dropout() for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

In [10]:
batch_size = 128
num_nodes = 1024 #number of nodes in the hidden layer

graph = tf.Graph()
with graph.as_default():

# Input data. For the training data, we use a placeholder that will be fed
# at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  def weight_variable(shape):
    initial = tf.truncated_normal(shape)
    return tf.Variable(initial)

  def bias_variable(shape):
    initial = tf.zeros(shape)
    return tf.Variable(initial) 

  def nn_layer(input_tensor, input_dim, output_dim, act=tf.nn.relu):
    # Creates a simple neural net layer."""
    weights = weight_variable([input_dim, output_dim])
    biases = bias_variable([output_dim])
    preactivate = tf.matmul(input_tensor, weights) + biases
    activations = act(preactivate)
    return weights, biases, activations
    
  weights1, biases1, hidden1 = nn_layer(tf_train_dataset, image_size * image_size, num_nodes)
  '''Here lies the code for introducing Dropout'''
  keep_prob = tf.placeholder(tf.float32)
  dropped = tf.nn.dropout(hidden1, keep_prob)
  weights2, biases2, output = nn_layer(dropped, num_nodes, num_labels, act=tf.identity)

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=output)+
    beta * tf.nn.l2_loss(weights1) + beta * tf.nn.l2_loss(weights2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(output)

  valid_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_hidden, weights2) + biases2)

  test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  test_prediction = tf.nn.softmax(tf.matmul(test_hidden, weights2) + biases2)

In [11]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3606.052002
Minibatch accuracy: 9.4%
Validation accuracy: 27.6%
Minibatch loss at step 500: 21.351337
Minibatch accuracy: 85.2%
Validation accuracy: 83.5%
Minibatch loss at step 1000: 1.024052
Minibatch accuracy: 78.1%
Validation accuracy: 83.0%
Minibatch loss at step 1500: 0.648513
Minibatch accuracy: 84.4%
Validation accuracy: 83.3%
Minibatch loss at step 2000: 0.683707
Minibatch accuracy: 88.3%
Validation accuracy: 83.0%
Minibatch loss at step 2500: 0.786054
Minibatch accuracy: 81.2%
Validation accuracy: 82.5%
Minibatch loss at step 3000: 0.855118
Minibatch accuracy: 81.2%
Validation accuracy: 82.9%
Test accuracy: 89.7%



-- Now, extreme overfitting case

In [12]:
num_steps = 3001

''' we select a small subset from training dataset'''
train_dataset_2 = train_dataset[:500, :]
train_labels_2 = train_labels[:500]

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels_2.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset_2[offset:(offset + batch_size), :]
        batch_labels = train_labels_2[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step {}: {}".format(step, l))
            print("Minibatch accuracy: {:.1f}".format(accuracy(predictions, batch_labels)))
            print("Validation accuracy: {:.1f}".format(accuracy(valid_prediction.eval(), valid_labels)))
    print("Test accuracy: {:.1f}".format(accuracy(test_prediction.eval(), test_labels)))

Initialized
Minibatch loss at step 0: 3619.21679688
Minibatch accuracy: 7.8
Validation accuracy: 27.8
Minibatch loss at step 500: 21.1089248657
Minibatch accuracy: 100.0
Validation accuracy: 79.2
Minibatch loss at step 1000: 0.531735181808
Minibatch accuracy: 99.2
Validation accuracy: 79.5
Minibatch loss at step 1500: 0.325522065163
Minibatch accuracy: 100.0
Validation accuracy: 79.7
Minibatch loss at step 2000: 0.314867407084
Minibatch accuracy: 99.2
Validation accuracy: 79.6
Minibatch loss at step 2500: 0.308524668217
Minibatch accuracy: 100.0
Validation accuracy: 79.6
Minibatch loss at step 3000: 0.307366847992
Minibatch accuracy: 99.2
Validation accuracy: 79.5
Test accuracy: 86.7


## Problem 4

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is 97.1%.

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

I am adding 2 more layers and implement the learning rate decay. You should be careful with the initial learning rate. It should be small enough to converge with such a seep structure. b is another parameter which can be played with.

In [13]:
batch_size = 128
beta = 0.001
num_nodes_1 = 1024
num_nodes_2 = 512 

graph = tf.Graph()
with graph.as_default():

# Input data. For the training data, we use a placeholder that will be fed
# at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

  def bias_variable(shape):
    initial = tf.zeros(shape)
    return tf.Variable(initial) 

  def nn_layer(input_tensor, input_dim, output_dim, act=tf.nn.relu):
    weights = weight_variable([input_dim, output_dim])
    biases = bias_variable([output_dim])
    preactivate = tf.matmul(input_tensor, weights) + biases
    activations = act(preactivate)
    return weights, biases, activations
  
  '''nn with 2 hidden layers'''
  weights1, biases1, hidden1 = nn_layer(tf_train_dataset, image_size * image_size, num_nodes_1)
  keep_prob = tf.placeholder(tf.float32)
  dropped1 = tf.nn.dropout(hidden1, keep_prob)
    
  weights2, biases2, hidden2 = nn_layer(dropped1, num_nodes_1, num_nodes_2, act=tf.identity)
  dropped2 = tf.nn.dropout(hidden2, keep_prob)
    
  weights3, biases3, output = nn_layer(dropped2, num_nodes_2, num_labels, act=tf.identity)

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=output) +
    beta * tf.nn.l2_loss(weights1) + 
    beta * tf.nn.l2_loss(weights2) +
    beta * tf.nn.l2_loss(weights3))
  
  # Optimizer.
  '''using learning rate decay'''
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.1, global_step, 100000, 0.96, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(output)

  valid_hidden1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  valid_hidden2 = tf.nn.relu(tf.matmul(valid_hidden1, weights2) + biases2)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_hidden2, weights3) + biases3)

  test_hidden1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  test_hidden2 = tf.nn.relu(tf.matmul(test_hidden1, weights2) + biases2)
  test_prediction = tf.nn.softmax(tf.matmul(test_hidden2, weights3) + biases3)

In [14]:
num_steps = 8001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 19.009058
Minibatch accuracy: 10.2%
Validation accuracy: 33.4%
Minibatch loss at step 500: 5.407803
Minibatch accuracy: 77.3%
Validation accuracy: 81.9%
Minibatch loss at step 1000: 4.910588
Minibatch accuracy: 80.5%
Validation accuracy: 82.6%
Minibatch loss at step 1500: 4.212258
Minibatch accuracy: 85.9%
Validation accuracy: 83.8%
Minibatch loss at step 2000: 3.847483
Minibatch accuracy: 89.1%
Validation accuracy: 84.5%
Minibatch loss at step 2500: 3.560848
Minibatch accuracy: 84.4%
Validation accuracy: 84.9%
Minibatch loss at step 3000: 3.283221
Minibatch accuracy: 82.8%
Validation accuracy: 85.5%
Minibatch loss at step 3500: 3.030287
Minibatch accuracy: 82.8%
Validation accuracy: 85.7%
Minibatch loss at step 4000: 2.729518
Minibatch accuracy: 85.9%
Validation accuracy: 86.1%
Minibatch loss at step 4500: 2.524306
Minibatch accuracy: 85.2%
Validation accuracy: 86.1%
Minibatch loss at step 5000: 2.253728
Minibatch accuracy: 90.6%
Validation accura